# Session 3b - Inmas Workshop Machine Learning Workshop, January 13-14, 2024

Instructor: Christian Kuemmerle - kuemmerle@uncc.edu

**This version of the notebook is more suitable for students with more experience in machine learning / who are more familiar with coding/ some of the covered context. It contains fewer hints compared to the version ``session3b_PCA.ipynb``.**

## Principal Component Analysis

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np;  
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets

## Iris Dataset

We start by loading the [Iris dataset](https://en.wikipedia.org/wiki/Iris_flower_data_set) which is a well-known example dataset for classification problems (introduced by the statistician [Robert Fisher](https://en.wikipedia.org/wiki/Ronald_Fisher)). The dataset, which is [available in scikit-learn](https://scikit-learn.org/stable/auto_examples/datasets/plot_iris_dataset.html), consists of three different types of irises’ (Setosa, Versicolour, and Virginica) petal and sepal length and width, stored in a 150x4 numpy.ndarray.

In [ ]:
iris = datasets.load_iris()
X = iris.data
y = iris.target
print("Dimensions of X:",X.shape)
print("Dimensions of y:",y.shape)

Let's understand the dataset a little bit. First, we read the names of the categories ("targets").

In [ ]:
print (iris['target_names'])

Also, a detailed description of the dataset is provided.

In [ ]:
print(iris.DESCR)

We further explore that data a bit.

In [ ]:
print(iris.feature_names) 
# have a peek at feature variables
print (iris.data[:5])
# have a peek at target variables
print (iris.target[:5])

We check that indeed, the three classes are balanced in the sense that they each have 50 data samples:

In [ ]:
print("Number of samples in first iris class: %i" % (iris.target==0).sum())
print("Number of samples in second iris class: %i" % (iris.target==1).sum())
print("Number of samples in thrid iris class: %i" % (iris.target==2).sum())

Using seaborn and the visualization techniques we learnt in the December workshop (revisit notebook `02a_data_visualization_with_seaborn.ipynb` from that workshop), we can visualize pairwise scatter plots of the features. We use the class labels to color the data points.

In [ ]:
# this is a small loop for creating a list with the category names 
iris.target_class = []
for k in range(len(iris.target)):
    iris.target_class.append(iris.target_names[iris.target[k]])

In [ ]:
# here, we create a pandas.DataFrame from the data has this facilities a nice interfacing with seaborn
df_iris = pd.DataFrame(iris.data,columns=iris['feature_names'])
df_iris = df_iris.assign(iris_class=iris.target_class)

A seaborn pairplot is the tool we would like to use for that purpose.

In [ ]:
import seaborn as sns
sns.pairplot(df_iris, hue= "iris_class", palette="tab10", )
fig = plt.gcf()
fig.set_size_inches(20, 20)

While this "matrix" of pairwise scatterplots gives us accurate representations and visualizations of all the details of the dataset, it necessitates a lot of different plots to represent accurately the relationships of the four different feature variables (petal length, petal width, sepal length and sepal width). <br>

However, it is possible to visualize the data samples in their entirety with one plot?

Using the concept of _dimensionality reduction_ there is something we can do. We want to know, can we summarize the data in a way that dropping some of the dimensions only incurs a small inaccuracies, while enabling us to visualize the data?

Recall from linear algebra that the rank-$d$ singular value decomposition of a matrix M is the rank $d$ matrix $\hat M_d$ such that $\|M - \hat M_d \|_F$ is minimized. The corresponding singular vectors in $\mathbb{R}^D$ are called the (principal) components, while the singular vectors in $\mathbb{R}^N$ are called the loadings---the process of generating and analyzing this data is called Principal Component Analysis (PCA, for mathematicians, [Chapter 10 of this book](https://mml-book.github.io/book/mml-book.pdf) is a great resource).

Let's call the [PCA functionalities of scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) and see how well we can do in just two or three dimensions.

In [ ]:
from sklearn.decomposition import PCA

pca = PCA()
X_pca = pca.fit_transform(X.reshape(len(X),-1))
print('Explained variance of top 3 components: ',pca.explained_variance_ratio_[:3])
cumulative_variance_ratio = 100*np.cumsum(pca.explained_variance_ratio_)
print('2 components captures %.4g%% of total variation.' % cumulative_variance_ratio[1])
print('3 components captures %.4g%% of total variation.' % cumulative_variance_ratio[2])
#print('99%% variance with %d components.'% 1+np.argmax(np.cumsum(pca.explained_variance_ratio_)>0.99))

Now we can project the data on the first two coordinates.

In [ ]:
plt.title('Projection of data onto first two principal components')
plt.scatter(*X_pca.T[:2])
plt.show()

Now let's color the points by class.

In [ ]:
plt.title('Projection of data onto first two principal components')
for lbl,i in [("Setosa", 0), ("Versicolour", 1), ("Virginica", 2)]:
    plt.scatter(*X_pca.T[:2,y==i], label=lbl, s=10, alpha=1)
plt.legend()
plt.show()

At this point, please go back to the pairwise scatterplot visualizations we made above using seaborn.

**_In which sense is this visualization fundamentally more powerful and generalizable_ than what we did above?** <br>**Discuss with your group.**

Answer:

We have seen that the first 3 principal components capture more than 99% of the variance. Using a 3D plot, we can also visualize the projection of the data onto the first 3 principal components.

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure(1, figsize=(8, 8))
ax = Axes3D(fig, rect=[0, 0, 0.95, 1], elev=48, azim=134)
plt.axes(ax)
ax.set_title('Projection of data onto first three principal components')
for lbl,i in [("Setosa", 0), ("Versicolour", 1), ("Virginica", 2)]:
    ax.scatter(*X_pca.T[:3,y==i], label=lbl)
plt.legend()
plt.show()

## Fashion MNIST

Now we will consider the fashion MNIST dataset which we have also worked with in [Session 2](https://webpages.charlotte.edu/~ckuemme1/teaching/machinelearningworkshop2023/02-classification-nlp.html). We start by loading the data and visualising some examples.

In [ ]:
import torchvision
import torch
import torchvision.transforms as transforms
train_set = torchvision.datasets.FashionMNIST(root="./", download=True, 
                                              train=True,
                                              transform=transforms.Compose([transforms.ToTensor()]))

test_set = torchvision.datasets.FashionMNIST(root="./", download=True, 
                                              train=False,
                                              transform=transforms.Compose([transforms.ToTensor()]))

X_train = train_set.data.numpy()
Y_train = train_set.targets.numpy()

X_test = test_set.data.numpy()
Y_test = test_set.targets.numpy()

label_dict= {
0 : "tshirt",
1 : "pants",
2 : "sweater",
3 : "dress",
4 : "long sleeve",
5 : "sandal",
6 : "jacket",
7 : "sneaker",
8 : "bag",
9 : "shoe"}


n_row = 3
n_col = 5
fsize = (12,10)

fig, axs = plt.subplots(n_row, n_col, figsize=fsize)
imgs = [ np.random.choice(np.argwhere(Y_train == i).flatten()) for i in range(10)]
for i, ax, img in zip(label_dict, axs.ravel(), imgs):
    ax.set_title('%d: %s' % (i, label_dict[i]))
    ax.set_xticks([])
    ax.set_yticks([])
    ax.imshow(X_train[img], cmap='gray' )

for ax in axs.ravel()[len(label_dict):]:
    ax.set_visible(False)
plt.show()

In the previous classification tasks, we've had two main concerns with our datasets:
* The dimension was a little too big, making running the algorithms (particularly K-NN) somewhat tedious
* The class labels were a little iffy, particularly in the fashion dataset (are jackets and sweaters really distinguishable in these images?)

Let's see if we can use PCA to throw away some number of dimensions without loss of classification accuracy. 

**Task: Implement PCA (using scikit-learn) on this dataset and compute:**
- The explained variance of the top 5 components 
- The number of components that capture more than 99% of the variance

In [ ]:
### Write your code here
pca = 
X_pca =

When you have done that, you will see that we can certainly explain much more than $\frac{2}{784}$ of the variance in the data by looking at the top two principal components (this is what we would expect by just dropping all but 2 of the $784=28*28$ features). 

Moreover, there we will see that there is a matrix with rank signficantly less than 784 that approximates our data matrix to within 1% in norm.

Now let's look at the projection of the data onto the first two components.

In [ ]:
plt.title('Projection of data onto first two coordinates')
plt.scatter(*X_pca.T[:2])
plt.show()

Of course, it looks somewhat like a blob right now, so let's color the points by class.

In [ ]:
plt.title('Projection of data onto first two coordinates')
for i,lbl in label_dict.items():
    plt.scatter(*X_pca.T[:2,Y_train==i], label=lbl, s=10, alpha=0.5)
plt.legend()
plt.show()

We can see that the points corresponding to each class do seem to be grouped together in this representation, but it's still a little difficult to see. Let's focus on two classes:

In [ ]:
class1 = 0 # Change these to view different classes
class2 = 1
plt.title('Projection of data onto first two coordinates')
plt.scatter(*X_pca.T[:2,Y_train==class1], label=label_dict[class1])
plt.scatter(*X_pca.T[:2,Y_train==class2], label=label_dict[class2])
plt.legend()
plt.show()

Depending on the classes you're looking at, you might see a clear difference in where the samples of the two classes lie in this 2D representation.

## Exercise

Now let's see if we can revisit what we did yesterday and combine with our newly learnt technique for dimension reduction. We recall that when we tried to fit the classification models to this Fashion MNIST data, computational challenges arose and it took a considerable amount of time to work with some of the classifiers (even after subsampling of the pixels etc.).

We intend explore whether we can obtain accurate predictions using features obtained by the projection provided by PCA on only few principal components. In this exercise, the goal is to implement the following: 
- Pick your favorite classification method among k-nearest neighbors, logistic regression and a support vector classifier (we did not cover the latter in the workshop; however, it is another popular supervised learning method, see for example [the explanations in the scikit-learn documentation](https://scikit-learn.org/stable/modules/svm.html)).
- Run this classifier on the first 2, 5, 10 and 20 components of the data. 
- Compute the total variance captured by each number of components and plot the training and validation accuracies as a function of the number of components. 
- Make sure to compute the time it takes to run each model. If you have time left, you can run the classifier without dimensionality reduction to compare the running times of all implementations.
- (Optional/Homework) Try the two other classification methods, and compare which result in the best prediction accuracy for a dimensionality reduction of fixed dimension.

In [ ]:
### Write your code here